In [1]:
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM,FcM_like
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
from berny import Berny, geomlib,Geometry,Math
import berny.coords
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from Morse import mpa
from AP_class import APDFT_perturbator as APP
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [3]:
def nr_step(ic,g,h):
    ncoords=ic.shape[0]*ic.shape[1]
    p1cds=ic-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1)).reshape(ic.shape[0],ic.shape[1])
    p1cds-=p1cds[0]
    print (norm(p1cds[1]))
    if p1cds.shape[0]>2:
        print (anglecenter(p1cds),anglecenter(p1cds)*np.pi/180)
    return(p1cds)

In [4]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic
tbbde={"CH":105/627.5,"HC":105/627.5,"NH":110/627.5,"HN":110/627.5,\
       "OH":119/627.5,"HO":119/627.5,"FH":136/627.5,"HF":136/627.5}
def mpa_pb(coords,atoml,g,h,gic=False,solve_ic=False,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
    print(g_ic)
    h_ic=build_h_ic(s,g_ic,h,B_inv)
    #print (s.coords._coords)
    if not solve_ic:    
        return mpa(g_ic[0],h_ic[0,0],q0[0],tbbde[atoml[s.coords._coords[0].i]+atoml[s.coords._coords[0].j]])[0]\
                +g_ic[0]/h_ic[0,0]-q0[0]
    bnr=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                bondatoms=atoml[s.coords._coords[i].i]+atoml[s.coords._coords[i].j]
                ddq_mb[i]+=mpa(g_ic[i],h_ic[i,i],q0[i],tbbde[bondatoms])[0]+g_ic[i]/h_ic[i,i]-q0[i]
    dq_NR=-np.linalg.solve(h_ic,g_ic)
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    return q0,dq_NR,q0+dq_NR,q0+dq_NR+ddq_mb

In [5]:
nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy")

In [6]:
nh3lst=[]
for x in range(4):
    nh3lst.append("NHHH"[x])
    nh3lst.extend(nh3optg[x].tolist())    

In [7]:
nh3srt=(("{} "*4+";")*4)[:-1].format(*nh3lst)

In [8]:
NH3=gto.M(atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf = scf.RHF(NH3)
mf.scf()
g0=mf.Gradients().grad()
h0=mf.Hessian().hess()
NH3coords=NH3.atom_coords()

converged SCF energy = -56.2221835947892
--------------- RHF gradients ---------------
         x                y                z
0 N    -0.0000000000     0.0000180258    -0.0001418121
1 H     0.0000000000     0.0000355116     0.0000167973
2 H    -0.0000802381    -0.0000267687     0.0000625074
3 H     0.0000802381    -0.0000267687     0.0000625074
----------------------------------------------


In [9]:
afN=aaff(mf,[1,0,0,0])+alc_deriv_grad_nuc(NH3,[1,0,0,0])
afH1=aaff(mf,[0,0,0,-1])+alc_deriv_grad_nuc(NH3,[0,0,0,-1])

In [10]:
af_1=afN+afH1
af_1

array([[-0.05145068, -0.04136139, -0.03686892],
       [-0.01657235,  0.01318325,  0.06197958],
       [ 0.04283018,  0.02991696, -0.03954284],
       [ 0.02519284, -0.00173881,  0.01443217]])

In [11]:
af_1[0]+=af_1[3]
af_1[3]-=af_1[3]
af_1

array([[-0.02625783, -0.04310021, -0.02243675],
       [-0.01657235,  0.01318325,  0.06197958],
       [ 0.04283018,  0.02991696, -0.03954284],
       [ 0.        ,  0.        ,  0.        ]])

In [12]:
nr_step(NH3coords,af_1,h0)

225.739350786074
28.608 0.49930379241053774


array([[   0.        ,    0.        ,    0.        ],
       [ -95.05248773,  201.07404107,  -38.63300504],
       [ -33.34037338,   36.38047156,  -32.92013828],
       [ -36.46263669, -200.82270892, -142.78504398]])

In [13]:
norm(oh2optg[1]),anglecenter(oh2optg)

(1.7756053665813991, 106.393)

In [14]:
norm(NH3coords[1])

1.8856061076490165

In [15]:
dl=.1
NH3_p=FcM(fcs=[dl,0,0,-dl],atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf_p = scf.RHF(NH3_p)
mf_p.scf(dm0=mf_p.init_guess_by_1e())
h0_p=mf_p.Hessian().hess()
af_p=aaff(mf_p,[1,0,0,-1])+alc_deriv_grad_nuc(NH3_p,[1,0,0,-1])

converged SCF energy = -57.9799515117373


In [16]:
NH3_m=FcM(fcs=[-dl,0,0,dl],atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf_m = scf.RHF(NH3_m)
mf_m.scf(dm0=mf_m.init_guess_by_1e())
h0_m=mf_m.Hessian().hess()
af_m=aaff(mf_m,[1,0,0,-1])+alc_deriv_grad_nuc(NH3_m,[1,0,0,-1])

converged SCF energy = -54.5169412230639


In [17]:
af_2=af_1+(af_p-af_m)/2/dl/2
af_3=af_1+(af_p-af_m)/2/dl/2+(af_p-2*af_1+af_m)/6/dl**2
ah_1=h0+(h0_p-h0_m)/2/dl
ah_2=h0+(h0_p-h0_m)/2/dl+(h0_p-2*h0+h0_m)/2/dl**2
af_2[0]+=af_2[3]
af_2[3]-=af_2[3]
af_3[0]+=af_3[3]
af_3[3]-=af_3[3]

In [20]:
nr_step(nh3optg,af_3,ah_2)

1.7484327450963009
107.161 1.8703122797296434


array([[ 0.        ,  0.        ,  0.        ],
       [-0.03707925,  0.42658979,  1.69518829],
       [ 1.37034458,  0.82215075, -0.70894616],
       [-1.53665019,  0.79431216, -0.76821526]])

In [21]:
norm(oh2optg[1]),anglecenter(oh2optg)

(1.7756053665813991, 106.393)

In [22]:
mpa_pb(NH3coords[:3],["O","H","H"],af_2[:3],ah_2[:3,:3,::],solve_ic=True)

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)]
[0.05865689 0.05867785 0.01418449]


(array([1.88560611, 1.88534951, 1.88831623]),
 array([-0.14219389, -0.14210765, -0.03737472]),
 array([1.74341221, 1.74324186, 1.85094151]),
 array([1.7755351 , 1.7753401 , 1.83647071]))

In [23]:
1.83647071*180/np.pi

105.22202088239375

In [24]:
oh2lst=[]
for x in range(3):
    oh2lst.append("OHH"[x])
    oh2lst.extend(oh2optg[x].tolist())  
oh2srt=(("{} "*4+";")*3)[:-1].format(*oh2lst)

In [25]:
OH2=gto.M(atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mf = scf.RHF(OH2)
mf.scf()
G0=mf.Gradients().grad()
H0=mf.Hessian().hess()
OH2coords=OH2.atom_coords()
AF_1=aaff(mf,[1,0,-1])+alc_deriv_grad_nuc(OH2,[1,0,-1])

converged SCF energy = -76.0633193327808
--------------- RHF gradients ---------------
         x                y                z
0 O    -0.0000165406    -0.0000132324    -0.0000136197
1 H     0.0000057480     0.0000045984     0.0000071565
2 H     0.0000107925     0.0000086340     0.0000064632
----------------------------------------------


In [26]:
OH2p=FcM(fcs=[dl,0,-dl],atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mf = scf.RHF(OH2p)
mf.scf(dm0=mf.init_guess_by_1e())
AF_1p=aaff(mf,[1,0,-1])+alc_deriv_grad_nuc(OH2p,[1,0,-1])
H0p=mf.Hessian().hess()

converged SCF energy = -78.2241681991639


In [27]:
OH2m=FcM(fcs=[-dl,0,dl],atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mf = scf.RHF(OH2m)
mf.scf(dm0=mf.init_guess_by_1e())
AF_1m=aaff(mf,[1,0,-1])+alc_deriv_grad_nuc(OH2m,[1,0,-1])
H0m=mf.Hessian().hess()

converged SCF energy = -73.9580532577637


In [28]:
AF_2=AF_1+(AF_1p-AF_1m)/dl/2/2
AF_3=AF_1+(AF_1p-AF_1m)/2/dl/2+(AF_1p-2*AF_1+AF_1m)/6/dl**2
AH_1=H0+(H0p-H0m)/2/dl
AH_2=H0+(H0p-H0m)/2/dl+(H0p-2*H0+H0m)/2/dl**2
AF_2[0]+=AF_2[2]
AF_2[2]-=AF_2[2]
AF_3[0]+=AF_3[2]
AF_3[2]-=AF_3[2]

In [29]:
mpa_pb(OH2coords[:2],["F","H"],AF_2[:2],AH_2[:2,:2,:,:],solve_ic=True)

[Bond(0, 1, weak=0)]
[0.05309818]


(array([1.77560537]),
 array([-0.10561262]),
 array([1.66999274]),
 array([1.68776093]))

In [30]:
fhoptg

array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.69574139]])

In [31]:
nr_step(OH2coords[:2],AF_1[:2],H0[:2,:2,:,:])

1.6965579106093105


array([[ 0.        ,  0.        ,  0.        ],
       [-0.41273035, -0.33018417,  1.61212308]])